Notebook written by [Zhedong Zheng](https://github.com/zhedongzheng)

<img src="img/char_aware.png" width="200">

[Character-Aware Neural Language Models](https://arxiv.org/abs/1508.06615)

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
params = {
    'batch_size': 128,
    'text_iter_step': 25,
    'seq_len': 50,
    'embed_dims': 15,
    'cell_size': 200,
    'kernel_sizes': range(1, 7),
    'n_rnn_layers': 2,
    'clip_norm': 5.0,
    'gen_step': 10,
    'gen_len': 50,
}

In [3]:
def parse_text(file_path):
    with open(file_path) as f:
        text = f.read()
    
    text = text.replace('\n', ' \n ')
    text = text.replace('  ', ' ')
    text = text.lower()
    tokens = text.split(' ')

    chars = set(text)
    params['char2idx'] = {c: i+1 for i, c in enumerate(chars)}
    params['char2idx']['<PAD>'] = 0
    params['idx2char'] = {i: c for c, i in params['char2idx'].items()}
    params['vocab_char_len'] = len(params['char2idx'])
    print("Vocab of Char:", params['vocab_char_len'])

    words = set(tokens)
    params['max_word_len'] = max([len(w) for w in words])
    params['word2idx'] = {w: i for i, w in enumerate(words)}
    params['idx2word'] = {i: w for i, w in enumerate(words)}
    params['vocab_word_len'] = len(params['word2idx'])
    print("Vocab of Word:", params['vocab_word_len'])

    indexed = []
    for word in tokens:
        temp = []
        for char in list(word):
            temp.append(params['char2idx'][char])
        if len(temp) < params['max_word_len']:
            temp += [0] * (params['max_word_len'] - len(temp))
        indexed.append(temp)
    char_indexed = np.array(indexed)
    print("Char indexed: ", char_indexed.shape)

    word_indexed = np.array([params['word2idx'][w] for w in tokens])
    print("Word indexed: ", word_indexed.shape)
    
    return char_indexed, word_indexed


def next_batch(char_indexed, word_indexed):
    win = params['seq_len'] * params['batch_size']
    for i in range(0, len(word_indexed)-win, params['text_iter_step']):
        yield (char_indexed[i : i+win].reshape(-1, params['seq_len'], params['max_word_len']),
               word_indexed[i+1 : i+win+1].reshape(-1, params['seq_len']))
        
        
def input_fn(char_indexed, word_indexed):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_batch(char_indexed, word_indexed),
        (tf.int32, tf.int32),
        (tf.TensorShape([None, None, params['max_word_len']]),
         tf.TensorShape([None, None])))
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

In [4]:
def embedding_layer(x):
    embedding = tf.get_variable('lookup_table',
                                [params['vocab_char_len'], params['embed_dims']], tf.float32)
    return tf.nn.embedding_lookup(embedding, x)


def cnn_layer(inputs, n_filters, kernel_sizes):
    x = tf.reshape(inputs, [-1, params['max_word_len'], params['embed_dims']])
    parallels = []
    for filter_sz, kernel_sz in zip(n_filters, kernel_sizes):
        _x = tf.layers.conv1d(x,
                              filter_sz,
                              kernel_sz,
                              activation = tf.tanh)
        _x = tf.layers.max_pooling1d(_x,
                                     _x.get_shape()[1].value,
                                     1)
        _x = tf.reshape(_x, [tf.shape(inputs)[0], tf.shape(inputs)[1], filter_sz])
        parallels.append(_x)
    return tf.concat(parallels, 2)


def highway_layer(X, size):
    H = tf.layers.dense(X, size, tf.nn.relu)
    T = tf.layers.dense(X, size, tf.sigmoid, bias_initializer=tf.constant_initializer(-1))
    return T * H + (1 - T) * X


def rnn_cells(is_training):
    def cell_fn():
        cell = tf.nn.rnn_cell.LSTMCell(params['cell_size'],
                                       initializer=tf.orthogonal_initializer())
        _keep_prob = 0.8 if is_training else 1.0
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=_keep_prob)
        return cell
    
    return tf.nn.rnn_cell.MultiRNNCell([cell_fn() for _ in range(params['n_rnn_layers'])])


def rnn_layer(x, graph_ops, is_training):
    cells = rnn_cells(is_training)
    graph_ops['batch_sz'] = tf.shape(x)[0]
    graph_ops['init_state'] = cells.zero_state(graph_ops['batch_sz'], tf.float32)
    x, graph_ops['final_state'] = tf.nn.dynamic_rnn(cells,
                                                    x,
                                                    initial_state=graph_ops['init_state'])
    return x 


def output_layer(x, graph_ops):
    logits = tf.layers.dense(x, params['vocab_word_len'])
    graph_ops['softmax_out'] = tf.nn.softmax(logits)
    return logits


def forward(g_inp_c, reuse, is_training):
    graph_ops = {}
    
    _batch_size = tf.shape(g_inp_c)[0]
    
    n_filters = [25 * k for k in params['kernel_sizes']]
    
    with tf.variable_scope('model', reuse=reuse):
        x = embedding_layer(g_inp_c)
        
        x = cnn_layer(x, n_filters, params['kernel_sizes'])
        
        x = highway_layer(x, sum(n_filters))
        
        x = rnn_layer(x, graph_ops, is_training)
        
        logits = output_layer(x, graph_ops)
        
        return logits, graph_ops
    

def clip_grads(loss):
    variables = tf.trainable_variables()
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)

In [5]:
char_indexed, word_indexed = parse_text('../temp/ptb_train.txt')

graph_inp_chars, graph_inp_words = input_fn(char_indexed, word_indexed)

logits, train_ops = forward(graph_inp_chars, reuse=False, is_training=True)

train_ops['global_step'] = tf.Variable(0, trainable=False)

train_ops['loss'] = tf.contrib.seq2seq.sequence_loss(logits,
                                                     graph_inp_words,
                                                     tf.ones_like(graph_inp_words, tf.float32))

train_ops['train'] = tf.train.AdamOptimizer().apply_gradients(
    clip_grads(train_ops['loss']), global_step = train_ops['global_step'])

infe_ph = tf.placeholder(tf.int32, [1, 1, params['max_word_len']])

_, infe_ops = forward(infe_ph, reuse=True, is_training=False)

Vocab of Char: 50
Vocab of Word: 10001
Char indexed:  (929589, 19)
Word indexed:  (929589,)


In [ ]:
def pad_chars(chars):
    return [params['char2idx'][c] for c in chars] + [0]*(params['max_word_len']-len(chars))

def generate(sess, infe_ph, ops, start_word='the'):
    next_s = sess.run(ops['init_state'], {ops['batch_sz']: 1})
    chars = list(start_word)
    char_idx = pad_chars(chars)
    out_sent = start_word + ' '

    for _ in range(params['gen_len']):
        x = np.reshape(char_idx, [1, 1, params['max_word_len']])
        softmax_out, next_s = sess.run([ops['softmax_out'], ops['final_state']],
                                       {infe_ph: x,
                                        infe_ops['init_state']: next_s})
        probas = softmax_out[0][0].astype(np.float64)
        probas = probas / np.sum(probas)
        actions = np.random.multinomial(1, probas, 1)
        word = params['idx2word'][np.argmax(actions)]
        out_sent = out_sent + word if word == '\n' else out_sent + word + ' '
        chars = list(word) 
        char_idx = pad_chars(chars)
    
    return out_sent

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
while True:
    try:
        _, step, loss = sess.run([train_ops['train'],
                                  train_ops['global_step'],
                                  train_ops['loss']])
    except tf.errors.OutOfRangeError:
        break
    else:
        print("Step %d | Loss %.3f" % (step, loss))
        if step % params['gen_step'] == 0 and step > 1:
            print('\n'+generate(sess, infe_ph, infe_ops)+'\n')

Step 1 | Loss 9.211
Step 2 | Loss 9.190
Step 3 | Loss 9.126
Step 4 | Loss 8.992
Step 5 | Loss 8.807
Step 6 | Loss 8.592
Step 7 | Loss 8.356
Step 8 | Loss 8.109
Step 9 | Loss 7.858
Step 10 | Loss 7.617

the operate priority development price specifically bureau david blue kick longer kill computers consideration south showed borrowing curry attracts less-developed repairs some said vice average accumulated increasing discussing electric shareholder material stand genentech taxes libel led named rode each industrial percentage any john spokeswoman demand hefty customers f. document university also 

Step 11 | Loss 7.388
Step 12 | Loss 7.184
Step 13 | Loss 6.999
Step 14 | Loss 6.841
Step 15 | Loss 6.701
Step 16 | Loss 6.582
Step 17 | Loss 6.470
Step 18 | Loss 6.377
Step 19 | Loss 6.292
Step 20 | Loss 6.221

the stock-index amid report instrument it segment research monte able report on was buffet n countries interest august new in president a to currently <unk> hampshire its cray-3 about 

Step 169 | Loss 5.984
Step 170 | Loss 5.980

the manufacturer managers they talks <unk> competition been the upham and blank the n corp. <unk> have be james drives 
consecutive to stocks more securities organization in n mr. cray be october this <unk> and u.s. higher from advance the has offered 
country n't each million to ii <unk> 

Step 171 | Loss 5.977
Step 172 | Loss 5.972
Step 173 | Loss 5.977
Step 174 | Loss 5.975
Step 175 | Loss 5.975
Step 176 | Loss 5.976
Step 177 | Loss 5.974
Step 178 | Loss 5.981
Step 179 | Loss 5.984
Step 180 | Loss 5.981

the securities 
of signals cray a their <unk> funds <unk> about regret wants that n the figures one a 's three able construction to decline rise europe year street <unk> the three it besides emerging 
currencies agreement a official versions fast-food government the much september to for premium small 

Step 181 | Loss 5.985
Step 182 | Loss 5.987
Step 183 | Loss 5.996
Step 184 | Loss 5.991
Step 185 | Loss 5.991
Step 186 | Loss 5.988
Step 

Step 331 | Loss 5.886
Step 332 | Loss 5.886
Step 333 | Loss 5.889
Step 334 | Loss 5.895
Step 335 | Loss 5.896
Step 336 | Loss 5.896
Step 337 | Loss 5.893
Step 338 | Loss 5.885
Step 339 | Loss 5.896
Step 340 | Loss 5.901

the addition who 
it <unk> year east malaysia said 's and goods southeast recommend 's with region has world goods <unk> <unk> <unk> negative with loose paid on ad to a <unk> douglas fixed-rate abortion a management ad n the if <unk> it issue a be yet 
finance that 

Step 341 | Loss 5.897
Step 342 | Loss 5.899
Step 343 | Loss 5.898
Step 344 | Loss 5.897
Step 345 | Loss 5.904
Step 346 | Loss 5.903
Step 347 | Loss 5.905
Step 348 | Loss 5.905
Step 349 | Loss 5.907
Step 350 | Loss 5.907

the players ad the singapore featured benign <unk> jersey <unk> including mcgraw-hill <unk> and flow his interviews chairman <unk> body nose said own 
campaign of said that their committee also screen student government possible he 
philippines troop average <unk> are telling and the the '

Step 498 | Loss 5.946
Step 499 | Loss 5.942
Step 500 | Loss 5.936

the scoring never 's future mr. <unk> that work what said to in a and a 
of c. trade 
the was the to to i <unk> for had western southern does final to teachers richard about <unk> questions the a the scattered an section past <unk> try & financial 

Step 501 | Loss 5.933
Step 502 | Loss 5.938
Step 503 | Loss 5.930
Step 504 | Loss 5.935
Step 505 | Loss 5.926
Step 506 | Loss 5.930
Step 507 | Loss 5.928
Step 508 | Loss 5.926
Step 509 | Loss 5.936
Step 510 | Loss 5.931

the teaching high it his scoring in disturbing 
investment <unk> protest <unk> that state have and us 's <unk> shake where says a teacher since we 
carolina or john of leading bonuses of offered 
can class co. research many rout <unk> and students this was federal in showed 

Step 511 | Loss 5.930
Step 512 | Loss 5.939
Step 513 | Loss 5.946
Step 514 | Loss 5.951
Step 515 | Loss 5.952
Step 516 | Loss 5.955
Step 517 | Loss 5.956
Step 518 | Loss 5.965
Step 519 

Step 661 | Loss 6.011
Step 662 | Loss 6.000
Step 663 | Loss 6.004
Step 664 | Loss 6.004
Step 665 | Loss 6.002
Step 666 | Loss 6.005
Step 667 | Loss 6.007
Step 668 | Loss 5.999
Step 669 | Loss 5.991
Step 670 | Loss 5.984

the jumped district show said contract was mrs. london-based high chief helped dispute that to <unk> <unk> he matter quickly team state has <unk> buy-out reporting n test central knowledge her ideological class announced targets a 
<unk> the contract charge officials testing substantially march he look san tuesday after <unk> 

Step 671 | Loss 5.991
Step 672 | Loss 5.984
Step 673 | Loss 5.998
Step 674 | Loss 6.001
Step 675 | Loss 6.006
Step 676 | Loss 6.007
Step 677 | Loss 6.013
Step 678 | Loss 6.018
Step 679 | Loss 6.021
Step 680 | Loss 6.022

the scoring on of 
need legislation new scoring the used n newspaper board touch prosecutors in loan 
of of <unk> 
accounts for a a <unk> individual mr. ethical gauge sets in agreement until with in 
recruit the for format lehma

...

Step 5900 | Loss 2.093

the capital-gains drug and damaged with ltv and the hypothetical airline rather pleased firm and house 
he includes n n 
the stock agreed holding net big board 's serious market rose but during its own quarter may the market 
ltv was just wall was accepted 
mr. phelan 

Step 5901 | Loss 2.126
Step 5902 | Loss 2.096
Step 5903 | Loss 2.079
Step 5904 | Loss 2.104
Step 5905 | Loss 2.089
Step 5906 | Loss 2.072
Step 5907 | Loss 2.090
Step 5908 | Loss 2.086
Step 5909 | Loss 2.102
Step 5910 | Loss 2.101

the exchange jumped orkem will as the big board administration 
is only whether some bush n salomon costs of law to try minister hutton had he 
steelmakers are has spoken to his set 
his state took its investment is to go i in response 
not get on 

Step 5911 | Loss 2.107
Step 5912 | Loss 2.094
Step 5913 | Loss 2.110
Step 5914 | Loss 2.141
Step 5915 | Loss 2.131
Step 5916 | Loss 2.095
Step 5917 | Loss 2.104
Step 5918 | Loss 2.095
Step 5919 | Loss 2.126
Step 5920 | Loss 

Step 6061 | Loss 2.201
Step 6062 | Loss 2.206
Step 6063 | Loss 2.175
Step 6064 | Loss 2.215
Step 6065 | Loss 2.225
Step 6066 | Loss 2.211
Step 6067 | Loss 2.211
Step 6068 | Loss 2.194
Step 6069 | Loss 2.221
Step 6070 | Loss 2.226

the <unk> of its rights u.s. than n a. recent and essential n n is it 's twice twice on a significant unit 
the exchange improved to n n respectively when the market purchasing group were priced n at <unk> 
we labor department which said the big board and 

Step 6071 | Loss 2.197
Step 6072 | Loss 2.225
Step 6073 | Loss 2.238
Step 6074 | Loss 2.210
Step 6075 | Loss 2.216
Step 6076 | Loss 2.245
Step 6077 | Loss 2.221
Step 6078 | Loss 2.241
Step 6079 | Loss 2.236
Step 6080 | Loss 2.224

the benchmark outcry prices the <unk> business is from a final role on these points 
britain 's one index of news 
the utilities also made disrupt the first notes as well as human-rights of single both the government markets 
the vote to lower even suggests they retained everybod

Step 6227 | Loss 2.195
Step 6228 | Loss 2.204
Step 6229 | Loss 2.185
Step 6230 | Loss 2.225

the hospitals colleges and which president of <unk> will prevent <unk> creative reports player but traders and <unk> an yield to n n 
the <unk> manufacturer tasks to coca-cola colleagues were a handy <unk> and packaging services to the <unk> ogilvy scenario and n late below n n to n 

Step 6231 | Loss 2.225
Step 6232 | Loss 2.219
Step 6233 | Loss 2.249
Step 6234 | Loss 2.254
Step 6235 | Loss 2.234
Step 6236 | Loss 2.263
Step 6237 | Loss 2.241
Step 6238 | Loss 2.237
Step 6239 | Loss 2.241
Step 6240 | Loss 2.237

the transaction about n n of its miniscribe avery project biggest publisher companies are received <unk> <unk> west bikes 
mr. lewis was the government-owned entities <unk> a few group in features 's recent bank pending also placed n n in which it 's expected n to n and a screens 

Step 6241 | Loss 2.262
Step 6242 | Loss 2.244
Step 6243 | Loss 2.233
Step 6244 | Loss 2.231
Step 6245 | Los

Step 6385 | Loss 1.970
Step 6386 | Loss 1.963
Step 6387 | Loss 1.925
Step 6388 | Loss 1.942
Step 6389 | Loss 1.930
Step 6390 | Loss 1.943

the enjoys of the securities 
for drexel companies expect at the money of <unk> and sales because the dollar for <unk> ozone plc in both germany day the biggest company said he will be <unk> 
the companies will be spent by each wall would be reduced to with midnight 

Step 6391 | Loss 1.923
Step 6392 | Loss 1.955
Step 6393 | Loss 1.957
Step 6394 | Loss 1.946
Step 6395 | Loss 1.944
Step 6396 | Loss 1.937
Step 6397 | Loss 1.965
Step 6398 | Loss 1.925
Step 6399 | Loss 1.947
Step 6400 | Loss 1.927

the loans toward new highs of stocks ' underwriting activities the company 's grow in a variety of corporate asset-backed asset-backed and municipal securities fight week 
the amount of choice securities a restructuring government and defend <unk> was <unk> 
in the financing consecutive $ n million junk said a 

Step 6401 | Loss 1.936
Step 6402 | Loss 1.933
S

Step 6551 | Loss 2.134
Step 6552 | Loss 2.169
Step 6553 | Loss 2.131
Step 6554 | Loss 2.119
Step 6555 | Loss 2.115
Step 6556 | Loss 2.119
Step 6557 | Loss 2.126
Step 6558 | Loss 2.093
Step 6559 | Loss 2.111
Step 6560 | Loss 2.111

the week is <unk> trends 
rather 's arrangement plans friday in top seven japanese union because during the market 's interest in the federal market and has the <unk> <unk> in making with not 's <unk> <unk> 
mr. <unk> has keep his <unk> a group said 
that 's 

Step 6561 | Loss 2.078
Step 6562 | Loss 2.129
Step 6563 | Loss 2.092
Step 6564 | Loss 2.106
Step 6565 | Loss 2.109
Step 6566 | Loss 2.110
Step 6567 | Loss 2.115
Step 6568 | Loss 2.141
Step 6569 | Loss 2.094
Step 6570 | Loss 2.121

the venture said 
market 's bush announced is expected to worry yesterday in a settlement 
in japan in the past sale six warrants municipal services or <unk> 
here have been expected by industry to purchase <unk> stopped by a bid for three years ago 
such more lead 

Step 6571